In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

# Reduce margins
display(HTML("<style>.prompt { min-width: 1ex !important; }</style>"))

In [298]:
import pandas as pd
import requests
import time
from functools import reduce

api_key = '658f4988855d67609e173b2316216105ea1be237'

# Get data
url_ACS1= f"https://api.census.gov/data/2020/acs/acs5/?get=B01001_001E,B01001_002E,B01001_026E,B06009_001E,B06009_002E,B06009_003E,B06009_004E,B06009_005E,B06009_006E&for=zip%20code%20tabulation%20area:*&key={api_key}"
url_ACS2= f"https://api.census.gov/data/2020/acs/acs5/?get=B08128_001E,B06010_001E,B06010_002E,B06010_003E,B06010_004E,B06010_005E,B06010_006E,B06010_007E,B06010_008E,B06010_009E,B06010_010E&for=zip%20code%20tabulation%20area:*&key={api_key}"
url_ACS3= f"https://api.census.gov/data/2020/acs/acs5/?get=B06010_011E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B18101_001E,B18101_006E,B18101_007E&for=zip%20code%20tabulation%20area:*&key={api_key}"
url_ACS4= f"https://api.census.gov/data/2020/acs/acs5/?get=B18101_009E,B18101_010E,B18101_025E,B18101_026E,B18101_028E,B18101_029E,B08128_061E,B27001_001E&for=zip%20code%20tabulation%20area:*&key={api_key}"

# To json
raw1 = requests.get(url_ACS1).json()
raw2 = requests.get(url_ACS2).json()
raw3 = requests.get(url_ACS3).json()
raw4 = requests.get(url_ACS4).json()

# To dataframe
df1 = pd.DataFrame(raw1[1:], columns=raw1[0])
df2 = pd.DataFrame(raw2[1:], columns=raw2[0])
df3 = pd.DataFrame(raw3[1:], columns=raw3[0])
df4 = pd.DataFrame(raw4[1:], columns=raw4[0])

# Compile the list of dataframes
data_frames = [df1, df2, df3, df4]

# Outer join
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['zip code tabulation area'],how='outer'), data_frames)

In [299]:
# Rename so that we can understand each variable
df_merged.rename(columns = {
    # Total population (Denominator)
    "B01001_001E": "total_population",
    "B01001_002E": "male_population",
    "B01001_026E": "female_population",
    
    # Education
    "B06009_001E": "edu_population",
     "B06009_002E": "edu_less_highshcool",
     "B06009_003E": "edu_highshcool",
     "B06009_004E": "edu_college",
     "B06009_005E": "edu_bachelor",
     "B06009_006E": "edu_graduate_professional",
 
    # Employment
    "B08128_001E": "employment_population",
    
    # Income 
     "B06010_001E": "income_population",
     "B06010_002E": "income_no",
     "B06010_003E": "income_less_10000",
     "B06010_004E": "income_10000_15000",
     "B06010_005E": "income_15000_25000",
     "B06010_006E": "income_25000_35000",
     "B06010_007E": "income_35000_45000",
     "B06010_008E": "income_45000_55000",
     "B06010_009E": "income_55000_65000",
     "B06010_010E": "income_65000_75000",
     "B06010_011E": "income_more_75000",

    # Race 
     "B02001_001E": "race_population",                  
     "B02001_002E": "race_white",  
     "B02001_003E": "race_african_american",  
     "B02001_004E": "race_asian",                  
     "B02001_005E": "race_native_american",  
     "B02001_006E": "race_others",                      
  
    # Disability 
     "B18101_001E": "disability_population",   
    
     "B18101_006E": "male_5to17_population",
     "B18101_007E": "disability_male_5to17",
     "B18101_009E": "male_18to34_population",
     "B18101_010E": "disability_male_18to34",

     "B18101_025E": "female_5to17_population",
     "B18101_026E": "disability_female_5to17",
     "B18101_028E": "female_18to34_population",
     "B18101_029E": "disability_female_18to34",
    
      # Work from home
     "B08128_061E": "WFH_population",  
    
      # Health insurnace
     "B27001_001E": "healthinsurance_population", 
    
      # Key variable 
     "zip code tabulation area": "zipcode"}, inplace=True)

# Formatting 
for col in list(df_merged.columns):
    if col is not 'zipcode':
        df_merged[col] = df_merged[col].astype(float)
    else:
        df_merged[col] = df_merged[col].astype(str)

In [301]:
# Preprocessing (covert them to ratio variables)
df_merged['male_ratio'] = (df_merged['male_population']/df_merged['total_population'])*100
df_merged['female_ratio'] = (df_merged['female_population']/df_merged['total_population'])*100
df_merged['edu_less_highshcool_ratio'] = (df_merged['edu_less_highshcool']/df_merged['edu_population'])*100
df_merged['edu_highshcool_ratio'] = (df_merged['edu_highshcool']/df_merged['edu_population'])*100
df_merged['edu_college_ratio'] = (df_merged['edu_college']/df_merged['edu_population'])*100
df_merged['edu_bachelor_ratio'] = (df_merged['edu_bachelor']/df_merged['edu_population'])*100
df_merged['edu_graduate_professional_ratio'] = (df_merged['edu_graduate_professional']/df_merged['edu_population'])*100
df_merged['employment_ratio'] = (df_merged['employment_population']/df_merged['total_population'])*100
df_merged['income_no_ratio'] = (df_merged['income_no']/df_merged['income_population'])*100
df_merged['income_less_10000_ratio'] = (df_merged['income_less_10000']/df_merged['income_population'])*100
df_merged['income_10000_15000_ratio'] = (df_merged['income_10000_15000']/df_merged['income_population'])*100
df_merged['income_15000_25000_ratio'] = (df_merged['income_15000_25000']/df_merged['income_population'])*100
df_merged['income_25000_35000_ratio'] = (df_merged['income_25000_35000']/df_merged['income_population'])*100
df_merged['income_35000_45000_ratio'] = (df_merged['income_35000_45000']/df_merged['income_population'])*100
df_merged['income_45000_55000_ratio'] = (df_merged['income_45000_55000']/df_merged['income_population'])*100
df_merged['income_55000_65000_ratio'] = (df_merged['income_55000_65000']/df_merged['income_population'])*100
df_merged['income_65000_75000_ratio'] = (df_merged['income_65000_75000']/df_merged['income_population'])*100
df_merged['income_more_75000_ratio'] = (df_merged['income_more_75000']/df_merged['income_population'])*100
df_merged['white_ratio'] = (df_merged['race_white']/df_merged['race_population'])*100
df_merged['race_african_american_ratio'] = (df_merged['race_african_american']/df_merged['race_population'])*100
df_merged['race_asian_ratio'] = (df_merged['race_asian']/df_merged['race_population'])*100
df_merged['race_native_american_ratio'] = (df_merged['race_native_american']/df_merged['race_population'])*100
df_merged['race_others_ratio'] = (df_merged['race_others']/df_merged['race_population'])*100
df_merged['disability_ratio'] = (df_merged['disability_population']/df_merged['total_population'])*100
df_merged['WFH_ratio'] = (df_merged['WFH_population']/df_merged['total_population'])*100
df_merged['healthinsurance_ratio'] = (df_merged['healthinsurance_population']/df_merged['total_population'])*100

In [302]:
# Keep columns needed
columns_needed = ['male_ratio','female_ratio','edu_less_highshcool_ratio','edu_highshcool_ratio','edu_college_ratio','edu_bachelor_ratio','edu_graduate_professional_ratio',
                  'employment_ratio','income_no_ratio','income_less_10000_ratio','income_10000_15000_ratio','income_15000_25000_ratio','income_25000_35000_ratio','income_35000_45000_ratio',
                  'income_45000_55000_ratio','income_55000_65000_ratio','income_65000_75000_ratio','income_more_75000_ratio',
                  'white_ratio','race_african_american_ratio','race_asian_ratio','race_native_american_ratio','race_others_ratio',
                  'disability_ratio','WFH_ratio','healthinsurance_ratio','zipcode']

df_merged = df_merged[columns_needed]
df_merged

,male_ratio,female_ratio,edu_less_highshcool_ratio,edu_highshcool_ratio,edu_college_ratio,edu_bachelor_ratio,edu_graduate_professional_ratio,employment_ratio,income_no_ratio,income_less_10000_ratio,...,income_more_75000_ratio,white_ratio,race_african_american_ratio,race_asian_ratio,race_native_american_ratio,race_others_ratio,disability_ratio,WFH_ratio,healthinsurance_ratio,zipcode
0,68.247248,31.752752,21.811269,40.131352,28.171448,5.599723,4.286208,29.833474,37.616331,62.383669,...,1.501051,20.265312,73.863957,0.762066,0.000000,0.000000,65.255433,2.257973,65.255433,29590
1,49.217993,50.782007,25.467785,27.779058,28.069407,11.747627,6.936123,38.046463,20.364278,79.635722,...,10.833803,69.813987,2.324486,1.625929,3.304350,0.239582,99.588134,1.686497,99.588134,93306
2,47.084762,52.915238,55.650224,23.318386,18.699552,2.331839,0.000000,29.348359,27.250000,72.750000,...,2.107143,54.654581,0.979912,0.000000,0.000000,0.000000,100.000000,1.028907,100.000000,93660
3,49.223553,50.776447,8.165153,16.641440,27.152708,28.632694,19.408005,47.664321,12.146158,87.853842,...,26.090410,76.326298,1.971224,1.819104,4.272042,0.000000,92.831337,8.075046,92.831337,93110
4,68.140379,31.859621,41.701481,29.085960,24.737309,3.259906,1.215344,18.976810,52.691384,47.308616,...,2.779408,57.678350,11.984422,1.473712,0.783324,0.101788,60.444326,0.690388,60.444326,93212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,53.079710,46.920290,8.816705,43.155452,35.498840,6.728538,5.800464,36.050725,10.288066,89.711934,...,6.790123,99.637681,0.000000,0.000000,0.000000,0.000000,100.000000,0.362319,100.000000,16623
33116,48.205855,51.794145,8.985879,55.391528,24.711168,8.664955,2.246470,45.467422,12.108108,87.891892,...,6.324324,93.484419,0.424929,0.094429,0.000000,0.000000,99.150142,4.060434,99.150142,16627
33117,48.888889,51.111111,17.695473,48.971193,24.691358,6.584362,2.057613,40.317460,6.273063,93.726937,...,4.059041,98.730159,0.000000,0.000000,1.269841,0.000000,100.000000,2.857143,100.000000,16634
33118,47.949081,52.050919,5.370370,52.222222,22.407407,14.629630,5.370370,37.340877,10.899654,89.100346,...,9.515571,99.151344,0.424328,0.000000,0.424328,0.000000,100.000000,3.960396,100.000000,16640


In [303]:
# to csv file
df_merged.to_csv(r'ACS.csv', index=False)